# **Forecast créditos generales**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

# 1. Cantidad

In [3]:
creditos_2019_pachuca = df_financiamientos_2019_puebla[['id','año','mes','modalidad','monto']]
creditos_2020_pachuca = df_financiamientos_2020_puebla[['id','año','mes','modalidad','monto']]
creditos_2021_pachuca = df_financiamientos_2021_puebla[['id','año','mes','modalidad','monto']]
creditos_2022_pachuca = df_financiamientos_2022_puebla[['id','año','mes','modalidad','monto']]
creditos_2023_pachuca = df_financiamientos_2023_puebla[['id','año','mes','modalidad','monto']]
creditos_2024_pachuca = df_financiamientos_2024_puebla[['id','año','mes','modalidad','monto']]

In [5]:
total_creditos_2019_pachuca = creditos_2019_pachuca.groupby('mes').count().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2020_pachuca = creditos_2020_pachuca.groupby('mes').count().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2021_pachuca = creditos_2021_pachuca.groupby('mes').count().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2022_pachuca = creditos_2022_pachuca.groupby('mes').count().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2023_pachuca = creditos_2023_pachuca.groupby('mes').count().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2024_pachuca = creditos_2024_pachuca.groupby('mes').count().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'num_registros']]


total_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_conteo_creditos_pachuca.head()

,año,mes,num_registros
0,2019,1,259
1,2019,2,250
2,2019,3,276
3,2019,4,258
4,2019,5,302


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [7]:
from sklearn.metrics import r2_score
import numpy as np

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.5507430690647255
Error cuadrático medio (MSE): 38745.15118667437
Error absoluto medio (MAE): 167.5549617998163
Raíz del error cuadrático medio (RMSE): 196.8378804668308


In [9]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
nuevas_fechas.head()

total_conteo_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_conteo_pachuca

,año,mes,num_registros
0,2019,1,259
1,2019,2,250
2,2019,3,276
3,2019,4,258
4,2019,5,302
...,...,...,...
99,2026,8,1046
100,2026,9,1047
101,2026,10,1048
102,2026,11,1049


In [10]:
total_conteo_pachuca = total_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_conteo_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales
# Formatear la columna 'montos' con la función personalizada
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].map(format_number)
total_conteo_pachuca

,año,num_registros,modalidad
0,2019,"3,448",Créditos (todos)
1,2020,"3,192",Créditos (todos)
2,2021,"3,481",Créditos (todos)
3,2022,"3,104",Créditos (todos)
4,2023,"14,156",Créditos (todos)
5,2024,"11,953",Créditos (todos)
6,2025,"10,994",Créditos (todos)
7,2026,"12,533",Créditos (todos)


In [11]:
# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 1]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 1]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 1]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 1]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 1]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

creditos_2024_pachuca_modalidad_1 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 1]
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'num_registros']]

nueva_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
nueva_conteo_creditos_pachuca.head()

,año,mes,num_registros
0,2019,1,105
1,2019,2,91
2,2019,3,102
3,2019,4,90
4,2019,5,129


In [12]:
# Preparar datos para el modelo
X = nueva_conteo_creditos_pachuca[['año', 'mes']]
y = nueva_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [13]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([nueva_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca['modalidad'] = 'Nueva'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo_pachuca['num_registros'] = nueva_conteo_pachuca['num_registros'].map(format_number)
nueva_conteo_pachuca

,año,num_registros,modalidad
0,2019,"1,300",Nueva
1,2020,"1,210",Nueva
2,2021,"1,248",Nueva
3,2022,"1,263",Nueva
4,2023,"6,019",Nueva
5,2024,"5,035",Nueva
6,2025,"4,683",Nueva
7,2026,"5,372",Nueva


In [14]:
# Filtrar los datos donde 'modalidad' sea igual a '2'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 2]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 2]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 2]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 2]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 2]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

creditos_2024_pachuca_modalidad_1 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 2]
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'num_registros']]

# Concatenar los DataFrames resultantes para cada año
mejoramiento_conteo_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca,total_creditos_2021_pachuca,total_creditos_2022_pachuca,total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
mejoramiento_conteo_pachuca.head()

,año,mes,num_registros
0,2019,1,64
1,2019,2,65
2,2019,3,70
3,2019,4,59
4,2019,5,60


In [15]:
# Preparar datos para el modelo
X = mejoramiento_conteo_pachuca[['año', 'mes']]
y = mejoramiento_conteo_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [16]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
mejoramiento_conteo_pachuca = pd.concat([mejoramiento_conteo_pachuca, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Mejoramientos'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,num_registros,modalidad
0,2019,753,Mejoramientos
1,2020,624,Mejoramientos
2,2021,653,Mejoramientos
3,2022,402,Mejoramientos
4,2023,"2,841",Mejoramientos
5,2024,"2,538",Mejoramientos
6,2025,"2,184",Mejoramientos
7,2026,"2,496",Mejoramientos


In [17]:
# Filtrar los datos donde 'modalidad' sea igual a '2'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 3]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 3]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 3]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 3]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 3]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

creditos_2024_pachuca_modalidad_1 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 3]
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'num_registros']]

# Concatenar los DataFrames resultantes para cada año
usada_conteo_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca,total_creditos_2021_pachuca,total_creditos_2022_pachuca,total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
usada_conteo_pachuca.head()

,año,mes,num_registros
0,2019,1,81
1,2019,2,81
2,2019,3,89
3,2019,4,93
4,2019,5,101


In [18]:
# Preparar datos para el modelo
X = usada_conteo_pachuca[['año', 'mes']]
y = usada_conteo_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [19]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
usada_conteo_pachuca = pd.concat([usada_conteo_pachuca, nuevas_fechas], ignore_index=True)
usada_conteo_pachuca = usada_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
usada_conteo_pachuca['modalidad'] = 'Usada'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
usada_conteo_pachuca['num_registros'] = usada_conteo_pachuca['num_registros'].map(format_number)
usada_conteo_pachuca

,año,num_registros,modalidad
0,2019,"1,191",Usada
1,2020,"1,114",Usada
2,2021,"1,279",Usada
3,2022,"1,175",Usada
4,2023,"4,285",Usada
5,2024,"3,549",Usada
6,2025,"3,312",Usada
7,2026,"3,740",Usada


In [20]:
# Filtrar los datos donde 'modalidad' sea igual a '2'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 4]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 4]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 4]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 4]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 4]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

creditos_2024_pachuca_modalidad_1 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 4]
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'num_registros']]

# Concatenar los DataFrames resultantes para cada año
otros_conteo_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca,total_creditos_2021_pachuca,total_creditos_2022_pachuca,total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
otros_conteo_pachuca.head()

,año,mes,num_registros
0,2019,1,9
1,2019,2,13
2,2019,3,15
3,2019,4,16
4,2019,5,12


In [21]:
# Preparar datos para el modelo
X = otros_conteo_pachuca[['año', 'mes']]
y = otros_conteo_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [22]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
otros_conteo_pachuca = pd.concat([otros_conteo_pachuca, nuevas_fechas], ignore_index=True)
otros_conteo_pachuca = otros_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
otros_conteo_pachuca['modalidad'] = 'Otros'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
otros_conteo_pachuca['num_registros'] = otros_conteo_pachuca['num_registros'].map(format_number)
otros_conteo_pachuca

,año,num_registros,modalidad
0,2019,204,Otros
1,2020,244,Otros
2,2021,301,Otros
3,2022,264,Otros
4,2023,"1,010",Otros
5,2024,830,Otros
6,2025,813,Otros
7,2026,927,Otros


In [23]:
conteo_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
    usada_conteo_pachuca,
    otros_conteo_pachuca,
])
# Datos
año = conteo_pachuca['año']
modalidades = conteo_pachuca['modalidad']
creditos = conteo_pachuca['num_registros']

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['num_registros'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))
fig.update_layout(
    title='Créditos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
#for trace in fig.data:
#    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == trace.name]
#    for i, point in enumerate(trace.y):
#        fig.add_annotation(
#            x=trace.x[i], 
#            y=point, 
#            text=f'{df_modalidad.iloc[i]["num_registros"]}', 
#            showarrow=False,
#            font=dict(color='black', size=10),
#            yshift=10
#        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_cantidad', carpeta='assets/graficas')
fig.show()

In [24]:
conteo_tres_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    usada_conteo_pachuca
])
# Datos
año = conteo_tres_pachuca['año']
modalidades = conteo_tres_pachuca['modalidad']
creditos = conteo_tres_pachuca['num_registros']

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = conteo_tres_pachuca[conteo_tres_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['num_registros'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))
fig.update_layout(
    title='Créditos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
for trace in fig.data:
    df_modalidad = conteo_tres_pachuca[conteo_tres_pachuca['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # No formatear el valor del monto como moneda
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{point}',  # Mostrar el valor simple del punto
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_cantidad', carpeta='assets/graficas')
fig.show()

## *DIRECTOS*

In [25]:
data_3 = {
    'Año': [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026],
    'Créditos (todos)': [3449,3192,	3482,	3104,	2058,	2309,	1531,	1015],
    'Vivienda Nueva': [1300,	1210,	1248,	1263,	812,	1126,	724,	1004],
    'Vivienda Usada': [1191,	1114,	1279,	1175,	810,	1189,	820,	1203],
}
df_3 = pd.DataFrame(data_3)

# Reestructurar los datos
trimestres = []
modalidades = []
montos = []

for index, row in df_3.iterrows():
    for col in df_3.columns[1:]:  # Excluimos la columna de Año
        trimestres.append(row['Año'])
        modalidades.append(col)
        montos.append(row[col])

# Crear el nuevo DataFrame
nuevo_df_3 = pd.DataFrame({
    'trimestres': trimestres,
    'modalidades': modalidades,
    'montos': montos
})
nuevo_df_3

,trimestres,modalidades,montos
0,2019,Créditos (todos),3449
1,2019,Vivienda Nueva,1300
2,2019,Vivienda Usada,1191
3,2020,Créditos (todos),3192
4,2020,Vivienda Nueva,1210
5,2020,Vivienda Usada,1114
6,2021,Créditos (todos),3482
7,2021,Vivienda Nueva,1248
8,2021,Vivienda Usada,1279
9,2022,Créditos (todos),3104


In [26]:
import plotly.graph_objects as go
import os
import plotly.io as pio
import locale

# Datos
trimestres = nuevo_df_3['trimestres']
modalidades = nuevo_df_3['modalidades']
montos = nuevo_df_3['montos']

fig = go.Figure()

# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = nuevo_df_3[nuevo_df_3['modalidades'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestres'],
        y=df_modalidad['montos'],
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Créditos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

for trace in fig.data:
    df_modalidad = nuevo_df_3[nuevo_df_3['modalidades'] == trace.name]
    for i, point in enumerate(trace.y):
        # No formatear el valor del monto como moneda
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{point}',  # Mostrar el valor simple del punto
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_cantidad_direct', carpeta='assets/graficas')
fig.show()

# 2. Total

### Base

In [28]:
creditos_2019_pachuca = df_financiamientos_2019_puebla[['id','año','mes','modalidad','monto']]
creditos_2020_pachuca = df_financiamientos_2020_puebla[['id','año','mes','modalidad','monto']]
creditos_2021_pachuca = df_financiamientos_2021_puebla[['id','año','mes','modalidad','monto']]
creditos_2022_pachuca = df_financiamientos_2022_puebla[['id','año','mes','modalidad','monto']]
creditos_2023_pachuca = df_financiamientos_2023_puebla[['id','año','mes','modalidad','monto']]
creditos_2024_pachuca = df_financiamientos_2024_puebla[['id','año','mes','modalidad','monto']]


In [29]:
total_creditos_2019_pachuca = creditos_2019_pachuca.groupby('mes').sum().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']]

total_creditos_2020_pachuca = creditos_2020_pachuca.groupby('mes').sum().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']]

total_creditos_2021_pachuca = creditos_2021_pachuca.groupby('mes').sum().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']]

total_creditos_2022_pachuca = creditos_2022_pachuca.groupby('mes').sum().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']]

total_creditos_2023_pachuca = creditos_2023_pachuca.groupby('mes').sum().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']]

total_creditos_2024_pachuca = creditos_2024_pachuca.groupby('mes').sum().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'montos']]

total_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_conteo_creditos_pachuca.head()

,año,mes,montos
0,2019,1,3.266928e+08
1,2019,2,2.851175e+08
2,2019,3,3.262253e+08
3,2019,4,2.926579e+08
4,2019,5,3.914894e+08


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [31]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.5731861527813825
Error cuadrático medio (MSE): 7.471123026178842e+16
Error absoluto medio (MAE): 237281786.25417086
Raíz del error cuadrático medio (RMSE): 273333551.29180247


In [32]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
total_todos_pachuca

,año,montos,modalidad
0,2019,"4,076,027,268.63",Créditos (todos)
1,2020,"3,864,621,305.63",Créditos (todos)
2,2021,"4,588,737,126.87",Créditos (todos)
3,2022,"4,432,531,506.29",Créditos (todos)
4,2023,"18,652,935,700.93",Créditos (todos)
5,2024,"15,701,339,813.10",Créditos (todos)
6,2025,"14,639,217,682.97",Créditos (todos)
7,2026,"16,749,193,105.49",Créditos (todos)


In [33]:
# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2024_pachuca_modalidad_1 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_nueva_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_creditos_nueva_pachuca.head()

,año,mes,montos
0,2019,1,1.984650e+08
1,2019,2,1.592060e+08
2,2019,3,1.865374e+08
3,2019,4,1.556628e+08
4,2019,5,2.394969e+08


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_creditos_nueva_pachuca[['año', 'mes']]
y = total_creditos_nueva_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [35]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.5223159217051608
Error cuadrático medio (MSE): 2.6710630315127228e+16
Error absoluto medio (MAE): 142051891.59380588
Raíz del error cuadrático medio (RMSE): 163433871.38267034


In [36]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_nueva = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_nueva)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_nueva['montos'] = predicciones
total_nueva_pachuca = pd.concat([total_creditos_nueva_pachuca, nuevas_fechas_nueva], ignore_index=True)
total_nueva_pachuca = total_nueva_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_nueva_pachuca['modalidad'] = 'Nueva'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_nueva_pachuca['montos'] = total_nueva_pachuca['montos'].map(format_number)
total_nueva_pachuca

,año,montos,modalidad
0,2019,"2,149,346,330.03",Nueva
1,2020,"1,835,917,525.71",Nueva
2,2021,"2,096,291,830.14",Nueva
3,2022,"2,095,463,445.66",Nueva
4,2023,"10,177,204,162.63",Nueva
5,2024,"8,412,214,850.50",Nueva
6,2025,"7,892,745,624.58",Nueva
7,2026,"9,066,812,255.62",Nueva


In [37]:
# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2019_pachuca_modalidad_3 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2020_pachuca_modalidad_3 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2021_pachuca_modalidad_3 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2022_pachuca_modalidad_3 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2023_pachuca_modalidad_3 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2024_pachuca_modalidad_3 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_usada_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_creditos_usada_pachuca.head()

,año,mes,montos
0,2019,1,9.470245e+07
1,2019,2,9.102607e+07
2,2019,3,1.150997e+08
3,2019,4,1.095701e+08
4,2019,5,1.388463e+08


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
# Preparar datos para el modelo
X = total_creditos_usada_pachuca[['año', 'mes']]
y = total_creditos_usada_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [39]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_usada = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_usada)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_usada['montos'] = predicciones
total_usada_pachuca = pd.concat([total_creditos_usada_pachuca, nuevas_fechas_usada], ignore_index=True)
total_usada_pachuca = total_usada_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_usada_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_usada_pachuca['montos'] = total_usada_pachuca['montos'].map(format_number)
total_usada_pachuca

,año,montos,modalidad
0,2019,"1,585,803,756.76",Usada
1,2020,"1,539,276,523.66",Usada
2,2021,"1,890,742,394.44",Usada
3,2022,"1,885,016,117.87",Usada
4,2023,"6,503,053,387.06",Usada
5,2024,"5,575,958,162.22",Usada
6,2025,"5,165,349,645.30",Usada
7,2026,"5,873,870,822.93",Usada


In [40]:
forecast_creditosgral_totales_pachuca = pd.concat([total_todos_pachuca, total_nueva_pachuca,total_usada_pachuca], ignore_index=True)
# Datos
trimestres = forecast_creditosgral_totales_pachuca['año']
modalidades = forecast_creditosgral_totales_pachuca['modalidad']
montos = forecast_creditosgral_totales_pachuca['montos']

# Limpiar las cadenas de montos y convertirlas a números
promedios_limpios = [float(montos.replace(',', '')) for montos in montos]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['montos'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Monto total anual',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["montos"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_total', carpeta='assets/graficas')
fig.show()

# 3. Promedio

### BASE

In [41]:
creditos_2019_pachuca = df_financiamientos_2019_puebla[['id','año','mes','modalidad','monto']]
creditos_2020_pachuca = df_financiamientos_2020_puebla[['id','año','mes','modalidad','monto']]
creditos_2021_pachuca = df_financiamientos_2021_puebla[['id','año','mes','modalidad','monto']]
creditos_2022_pachuca = df_financiamientos_2022_puebla[['id','año','mes','modalidad','monto']]
creditos_2023_pachuca = df_financiamientos_2023_puebla[['id','año','mes','modalidad','monto']]
creditos_2024_pachuca = df_financiamientos_2024_puebla[['id','año','mes','modalidad','monto']]

In [42]:
total_creditos_2019_pachuca = creditos_2019_pachuca.groupby('mes').mean().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']]

total_creditos_2020_pachuca = creditos_2020_pachuca.groupby('mes').mean().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']]

total_creditos_2021_pachuca = creditos_2021_pachuca.groupby('mes').mean().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']]

total_creditos_2022_pachuca = creditos_2022_pachuca.groupby('mes').mean().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']]

total_creditos_2023_pachuca = creditos_2023_pachuca.groupby('mes').mean().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']]

total_creditos_2024_pachuca = creditos_2024_pachuca.groupby('mes').mean().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2024
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'montos']]

total_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_conteo_creditos_pachuca.head()

,año,mes,montos
0,2019,1,1.261362e+06
1,2019,2,1.140470e+06
2,2019,3,1.181976e+06
3,2019,4,1.134333e+06
4,2019,5,1.296322e+06


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [44]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.22014095895854713
Error cuadrático medio (MSE): 25997726897.080708
Error absoluto medio (MAE): 123290.02204502794
Raíz del error cuadrático medio (RMSE): 161238.10621897265


In [48]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
total_todos_pachuca

,año,montos,modalidad
0,2019,"14,186,729.93",Créditos (todos)
1,2020,"14,362,017.99",Créditos (todos)
2,2021,"15,725,992.21",Créditos (todos)
3,2022,"17,013,046.50",Créditos (todos)
4,2023,"22,692,631.85",Créditos (todos)
5,2024,"20,606,750.41",Créditos (todos)
6,2025,"17,294,742.98",Créditos (todos)
7,2026,"17,792,420.34",Créditos (todos)


In [49]:
# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2024_pachuca_modalidad_1 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2023
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_nueva_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_creditos_nueva_pachuca.head()

,año,mes,montos
0,2019,1,1.890143e+06
1,2019,2,1.749516e+06
2,2019,3,1.828798e+06
3,2019,4,1.729587e+06
4,2019,5,1.856565e+06


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_creditos_nueva_pachuca[['año', 'mes']]
y = total_creditos_nueva_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [51]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): -0.0247056500112679
Error cuadrático medio (MSE): 45606697479.04817
Error absoluto medio (MAE): 155262.77012777346
Raíz del error cuadrático medio (RMSE): 213557.2463744749


In [52]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_nueva = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_nueva)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_nueva['montos'] = predicciones
total_nueva_pachuca = pd.concat([total_creditos_nueva_pachuca, nuevas_fechas_nueva], ignore_index=True)
total_nueva_pachuca = total_nueva_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_nueva_pachuca['modalidad'] = 'Nueva'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_nueva_pachuca['montos'] = total_nueva_pachuca['montos'].map(format_number)
total_nueva_pachuca

,año,montos,modalidad
0,2019,"19,984,445.30",Nueva
1,2020,"18,069,883.77",Nueva
2,2021,"19,950,859.84",Nueva
3,2022,"19,770,238.64",Nueva
4,2023,"33,587,713.26",Nueva
5,2024,"20,288,755.81",Nueva
6,2025,"20,477,156.30",Nueva
7,2026,"20,665,556.79",Nueva


In [53]:
# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2019_pachuca_modalidad_3 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2020_pachuca_modalidad_3 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2021_pachuca_modalidad_3 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2022_pachuca_modalidad_3 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2023_pachuca_modalidad_3 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2024_pachuca_modalidad_3 = creditos_2024_pachuca[creditos_2024_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2024_pachuca = creditos_2024_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2024_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2024_pachuca['año'] = 2023
total_creditos_2024_pachuca = total_creditos_2024_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_usada_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca,total_creditos_2024_pachuca], ignore_index=True)
total_creditos_usada_pachuca.head()

,año,mes,montos
0,2019,1,1.169166e+06
1,2019,2,1.123779e+06
2,2019,3,1.293255e+06
3,2019,4,1.178173e+06
4,2019,5,1.374716e+06


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
# Preparar datos para el modelo
X = total_creditos_usada_pachuca[['año', 'mes']]
y = total_creditos_usada_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [55]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_usada = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_usada)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_usada['montos'] = predicciones
total_usada_pachuca = pd.concat([total_creditos_usada_pachuca, nuevas_fechas_usada], ignore_index=True)
total_usada_pachuca = total_usada_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_usada_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_usada_pachuca['montos'] = total_usada_pachuca['montos'].map(format_number)
total_usada_pachuca

,año,montos,modalidad
0,2019,"15,870,259.30",Usada
1,2020,"16,374,604.30",Usada
2,2021,"17,690,403.55",Usada
3,2022,"18,920,696.34",Usada
4,2023,"31,013,214.89",Usada
5,2024,"19,204,675.49",Usada
6,2025,"19,853,609.31",Usada
7,2026,"20,502,543.14",Usada


In [56]:
forecast_creditosgral_totales_pachuca = pd.concat([total_todos_pachuca, total_nueva_pachuca,total_usada_pachuca], ignore_index=True)
# Datos
trimestres = forecast_creditosgral_totales_pachuca['año']
modalidades = forecast_creditosgral_totales_pachuca['modalidad']
montos = forecast_creditosgral_totales_pachuca['montos']

# Limpiar las cadenas de montos y convertirlas a números
promedios_limpios = [float(montos.replace(',', '')) for montos in montos]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['montos'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Monto total anual',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["montos"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_promedio', carpeta='assets/graficas')
fig.show()